In [29]:
import os
import glob
import pandas as pd
import numpy as np

In [55]:
import ast
csv_files = glob.glob('hashtags/*.{}'.format('csv'))

df_concat = pd.concat([pd.read_csv(f,usecols=(1,2),converters={2:ast.literal_eval}) for f in csv_files], ignore_index=True)

df_concat.to_csv('hashtags_merged.csv')

In [56]:
df = df_concat

# indexEmpty = df[ (df['hashtags'])].index

# df.drop(indexEmpty, inplace=True)
df = df[df.astype(str)['hashtags'] != '[]']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3330 entries, 0 to 3995
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   avatar    3330 non-null   object
 1   hashtags  3330 non-null   object
dtypes: object(2)
memory usage: 78.0+ KB


In [62]:
from wordsegment import load, segment
load()

def decomposed_hashtags(hashtags):
    flat_list = []
    for hashtag in hashtags:
        for segm in segment(hashtag):
            flat_list.append(segm)
    return list(set(flat_list))
    # return set()

df_decomposed = df.head(100)

df_decomposed['words'] = df_decomposed.apply(lambda row: decomposed_hashtags(row['hashtags']),axis=1)

df_decomposed
# df_100.to_csv('decomposed_hashtags_1000.csv')

/var/folders/nx/cjw5t0552msc_r1yfk4m77tr0000gn/T/ipykernel_71392/186257905.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_decomposed['words'] = df_decomposed.apply(lambda row: decomposed_hashtags(row['hashtags']),axis=1)


,avatar,hashtags,words
0,sinemunsal,"[#tbt, #pink, #Aydınlıkevler,, #tb]","[tbt, aydnlkevler, tb, pink]"
1,ellekorea,"[#ELLECoverstar, #제작비지원, #공유, #GONGYOO, #엘르5월호...","[elle, gong, yoo, up, xs, close, 5, j12, 3, st..."
2,ashanegi,"[#mentalbehene, #instareels, #reelitfeelit, #w...","[you, collaboration, bollywood, dancing, menta..."
3,portadosfundos,"[#BBB21., #cozinhacompimenta, #bbb, #bbb21, #b...","[nelly, os, coach, porta, jogo, do, are, comed..."
4,tobibakre,"[#WellnessWithJumia, #Camon16, #CBeyond, #Tobi...","[your, nigeria, is, 4mazfo, to, keep, trans, w..."
...,...,...,...
121,baranikster,"[#שירליוםראשון, #טונה, #🐟, #לפיההנחיות, #חייאת...",[]
122,movierapture,"[#Galaxyofentertainment, #cinephile, #movierec...","[cinema, gram, reviews, ram, is, entertainment..."
123,photo._.grammetry,"[#noodles, #reelsinstagram, #ketofy, #reelitfe...","[gram, super, is, veggie, new, go, savour, sha..."
124,hiddengemindia,"[#foodreels, #cakeofinstagram, #rasmalaicake, ...","[gram, cheese, try, ram, decorating, birthday,..."


In [63]:
import findspark
findspark.init()
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark                                 # only run after findspark.init()
from pyspark.sql import SparkSession
import urllib

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
spark_df = spark.createDataFrame(df_decomposed)

In [64]:
spark_df.toPandas()

,avatar,hashtags,words
0,sinemunsal,"[#tbt, #pink, #Aydınlıkevler,, #tb]","[tbt, aydnlkevler, tb, pink]"
1,ellekorea,"[#ELLECoverstar, #제작비지원, #공유, #GONGYOO, #엘르5월호...","[elle, gong, yoo, up, xs, close, 5, j12, 3, st..."
2,ashanegi,"[#mentalbehene, #instareels, #reelitfeelit, #w...","[you, collaboration, bollywood, dancing, menta..."
3,portadosfundos,"[#BBB21., #cozinhacompimenta, #bbb, #bbb21, #b...","[nelly, os, coach, porta, jogo, do, are, comed..."
4,tobibakre,"[#WellnessWithJumia, #Camon16, #CBeyond, #Tobi...","[your, nigeria, is, 4mazfo, to, keep, trans, w..."
...,...,...,...
95,baranikster,"[#שירליוםראשון, #טונה, #🐟, #לפיההנחיות, #חייאת...",[]
96,movierapture,"[#Galaxyofentertainment, #cinephile, #movierec...","[cinema, gram, reviews, ram, is, entertainment..."
97,photo._.grammetry,"[#noodles, #reelsinstagram, #ketofy, #reelitfe...","[gram, super, is, veggie, new, go, savour, sha..."
98,hiddengemindia,"[#foodreels, #cakeofinstagram, #rasmalaicake, ...","[gram, cheese, try, ram, decorating, birthday,..."


In [73]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import IDF, Tokenizer, StopWordsRemover, CountVectorizer, StringIndexer

spark_df = spark.createDataFrame(df.head(100))

wordsData = spark_df

decompose_hashtags_udf = udf(lambda row: decomposed_hashtags(row), ArrayType(StringType()))

filter_length_udf = udf(lambda row: [x for x in row if len(x) >= 3], ArrayType(StringType()))

wordsData = wordsData.withColumn('words',decompose_hashtags_udf(col('hashtags')))
wordsData =wordsData.withColumn('words_filtered',filter_length_udf(col('words')))

remover = StopWordsRemover(inputCol="words_filtered", outputCol = "filtered", locale="en_US")
wordsData = remover.transform(wordsData)
wordsData.toPandas()

cv = CountVectorizer(inputCol="filtered", outputCol="TF", vocabSize=1000) # keep the most 1000 common words
cvModel = cv.fit(wordsData)
wordsData = cvModel.transform(wordsData)

idf = IDF(inputCol="TF", outputCol="features")
idfModel = idf.fit(wordsData)
wordsData = idfModel.transform(wordsData)

wordsData.toPandas()

23/03/14 18:08:41 WARN StopWordsRemover: Default locale set was [en_RO]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


ERROR:root:KeyboardInterrupt while sending command.                 (0 + 0) / 8]
Traceback (most recent call last):
  File "/usr/local/opt/apache-spark/libexec/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/opt/apache-spark/libexec/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/crangaantonio/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt

KeyboardInterrupt

